In [13]:
from langchain_groq import ChatGroq

import os
from dotenv import load_dotenv
load_dotenv("secret.env")

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key = os.getenv("API_KEY"),

    # other params...
)

response = llm.invoke("Second person on the moon was ..")
print(response.content)

Edwin "Buzz" Aldrin was the second person to walk on the moon. He followed Neil Armstrong, who was the first person to walk on the moon, during the Apollo 11 mission in 1969.


In [14]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.nike.com/cdn-security-engineer-waf/job/R-48004")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























CDN Security Engineer (WAF)










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



In [15]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ###SCRAPED DATA FROM WEBSITE
        {page_data}
        ###INSTRUCTION
        The scraped text is from the career's page of a website
        Your job is to extract the job postings and return them in JSON format containing
        following keys: `role`, `experience`, `skills`, and `description`.
        only return the valid JSON.
        ###VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

{
  "role": "CDN Security Engineer (WAF)",
  "experience": {
    "required": "At least five (5) or more years of relevant experience",
    "desired": "Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers"
  },
  "skills": {
    "required": [
      "Strong foundation in Computer Science, software engineering, or a related field",
      "Proven track record of delivering high-quality solutions in a fast-paced, dynamic environment",
      "In-depth knowledge of Akamai's suite of tools and security offerings",
      "Strong understanding of API security requirements, web attack vectors, and threat indicators"
    ],
    "desired": [
      "Professional/functional credentials and certifications in cloud infrastructure, security, or a related field are highly desirable (e.g., AWS Certified Solutions Architect, CISSP, CEH)",
      "Cyber security experience is highly desired, but not required"
    ]
  },
  "description": {

In [16]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'CDN Security Engineer (WAF)',
 'experience': {'required': 'At least five (5) or more years of relevant experience',
  'desired': 'Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers'},
 'skills': {'required': ['Strong foundation in Computer Science, software engineering, or a related field',
   'Proven track record of delivering high-quality solutions in a fast-paced, dynamic environment',
   "In-depth knowledge of Akamai's suite of tools and security offerings",
   'Strong understanding of API security requirements, web attack vectors, and threat indicators'],
  'desired': ['Professional/functional credentials and certifications in cloud infrastructure, security, or a related field are highly desirable (e.g., AWS Certified Solutions Architect, CISSP, CEH)',
   'Cyber security experience is highly desired, but not required']},
 'description': {'job_summary': "As a CDN Security Engineer (WAF), you'll play 

In [17]:
type(json_res)

dict

In [2]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [ ]:
import chromadb,uuid

client = chromadb.PersistentClient('vectorstore')
#persistent client creates a folder to store your records
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas={"links":row['Links']},
                       ids=[str(uuid.uuid4())])

In [22]:
links = collection.query(query_texts=job['skills']['required'], n_results=2).get("metadatas",[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [ ]:
job = json_res
job['skills']['required']

'{\'role\': \'CDN Security Engineer (WAF)\', \'experience\': {\'required\': \'At least five (5) or more years of relevant experience\', \'desired\': \'Two (2) or more years of experience providing similar operations and sustainment support with Akamai or other CDN providers\'}, \'skills\': {\'required\': [\'Strong foundation in Computer Science, software engineering, or a related field\', \'Proven track record of delivering high-quality solutions in a fast-paced, dynamic environment\', "In-depth knowledge of Akamai\'s suite of tools and security offerings", \'Strong understanding of API security requirements, web attack vectors, and threat indicators\'], \'desired\': [\'Professional/functional credentials and certifications in cloud infrastructure, security, or a related field are highly desirable (e.g., AWS Certified Solutions Architect, CISSP, CEH)\', \'Cyber security experience is highly desired, but not required\']}, \'description\': {\'job_summary\': "As a CDN Security Engineer (W

In [24]:
prompt_email = PromptTemplate.from_template(
    """
        ###JOB DESCRIPTION
        {job_description}
        ###INSTRUCTION
        You are Srivatsav, a computer science student studying at Vellore Institute of Technology, Chennai.
        You are looking for internships to increase your experience and at the same time contribute towards the company.
        your job is to write a cold email to to the hr dept regarding the job mentioned above and fulfilling their needs.
        Also add the relevant ones from the follwing links to show your portfolio:{link_list}
        Remember, you are Srivatsav, 4th year student at VIT chennai.
        ###VALID JSON (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm 
res = chain_email.invoke(input={'job_description': str(job), 'link_list':links})
print(res.content)

Subject: Application for CDN Security Engineer (WAF) Internship at Nike

Dear Hiring Manager,

I am Srivatsav, a 4th-year student at the Vellore Institute of Technology, Chennai, pursuing a Bachelor's degree in Computer Science. I am excited to apply for the CDN Security Engineer (WAF) internship at Nike, as advertised on your website. With a strong foundation in computer science and a passion for cybersecurity, I am confident that I can contribute to Nike's digital presence and learn from the team.

As a student, I have been actively seeking opportunities to gain hands-on experience in the field. I have developed a strong understanding of computer science concepts, including software engineering, data structures, and algorithms. My experience with programming languages such as Python, Java, and JavaScript has equipped me with the skills to design, implement, and sustain cloud infrastructure and security posture.

I am particularly drawn to this role because of the opportunity to work 